In [253]:
import selenium
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
from selenium.webdriver.common.by import By
#headless means running chrome with chrome.exe
chrome_options.add_argument('--headless')

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

from selenium.webdriver.support.wait import WebDriverWait

import pandas as pd

In [254]:
driver = webdriver.Chrome('chromedriver',options=chrome_options) #this is how you call a web driver
review_driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [255]:
driver.get("https://www.beeradvocate.com/beer/top-rated/")
print(driver.title)

Top 250 Rated Beers | BeerAdvocate


In [256]:
data = pd.DataFrame()

### Task 1 - Scrapping reviews

In [257]:
beers = driver.find_elements(By.TAG_NAME,'tr')
for beer in beers[1:]:
    beer_name = beer.find_elements(By.TAG_NAME, 'td')[1].find_element(By.TAG_NAME, 'a').text
    beer_link = beer.find_elements(By.TAG_NAME, 'td')[1].find_element(By.TAG_NAME,'a').get_attribute('href')
    
    review_driver.get(beer_link)
    user_comments = review_driver.find_elements(By.CLASS_NAME, 'user-comment')
    
    for review in user_comments:
        try:
            user_rating = review.find_element(By.CLASS_NAME,'BAscore_norm').text
            user_review = review.find_elements(By.TAG_NAME, 'div')[3].text
            item = pd.DataFrame([[beer_name,review,user_rating]],columns = ['product_name','user_review','user_text'])
            data_df = pd.concat([data,item],axis=0,ignore_index = True)
        
        except:
            pass

In [258]:
data_df

,product_name,user_review,user_text
0,Expedition Stout - Bourbon Barrel-Aged,<selenium.webdriver.remote.webelement.WebEleme...,4.79


In [268]:
for i in user_comments:
    try: 
        user_rating2 = i.find_element(By.CLASS_NAME,'BAscore_norm')
    except:
        pass

In [269]:
user_rating2

<selenium.webdriver.remote.webelement.WebElement (session="7a3cb28d0e22f1bfde08d025593939ee", element="e9dc8c9a-eca6-4ef0-be15-c96edd452911")>

In [ ]:
driver.get(beer_page_hyperlinks_list[0])
user_comments = driver.find_elements_by_class_name('user-comment')
for comment in user_comments:
    user_rating = (comment.find_elements(By.CLASS_NAME, 'BAscore_norm'))
    user_review = (comment.find_elements(By.TAG_NAME,'div'))

user_rating

# for rating in user_rating:
#     for i in rating:
#         print(i.text)

In [47]:
beer_page_links2 = driver.find_element_by_tag_name("a") 

In [48]:
beer_page_links2

<selenium.webdriver.remote.webelement.WebElement (session="6d708c73c3f546a0fdac9d255021b9bc", element="cdd29811-f891-4fbe-a52b-98c8803decbb")>

### Task 2 - Calculating word frequencies

In [22]:
import pandas as pd
import nltk
#nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [23]:
beer_review_data = pd.read_csv('reviews_scraped.csv')

In [24]:
beer_review_data

,prod_name,rating,user_review
0,Prior 8,4.50,Had this on draft 9/8/22 @ the Hopleaf.\n\nMos...
1,Prior 8,4.46,Poured from a 11.2 oz bottle into a tulip glas...
2,Prior 8,2.45,"Look: Nice color, brown with a nice head\n\nSm..."
3,Prior 8,4.01,Had a 750mL leftover from St Bernardus day. Be...
4,Prior 8,4.35,Poured into a snifter a medium brown with a ta...
...,...,...,...
7373,Northern Hemisphere Harvest Wet Hop IPA,4.30,Look is dark orange/red with a good size head ...
7374,Northern Hemisphere Harvest Wet Hop IPA,4.27,-optically clear golden-amber bottle pour with...
7375,Northern Hemisphere Harvest Wet Hop IPA,4.34,SN never disappoints. This is one of their bes...
7376,Northern Hemisphere Harvest Wet Hop IPA,4.77,"De Labirratorium. En copa Teku. En almuerzo, c..."


In [25]:
#Tokenize words - remove stop words

tokenizer = nltk.RegexpTokenizer(r"\w+")

def to_lower(string1):
    return string1.lower()

def tokenize_the_text(string2):
    return tokenizer.tokenize(str(string2))

def remove_stop_words(string1):
    without_stopwords = []
    for word in string1:
        if word not in stopwords.words('english'):
            without_stopwords.append(word)
    return without_stopwords

beer_review_data['words_from_reviews'] = beer_review_data['user_review'].apply(to_lower).apply(tokenize_the_text).apply(remove_stop_words)

In [26]:
#Cal word Frequencies

all_words = []

for list1 in beer_review_data['words_from_reviews']:
    for word in list1:
        all_words.append(word)
        
word_freqs = pd.DataFrame(pd.DataFrame(all_words).rename(columns = {0 : 'word'}).value_counts()).reset_index().rename(columns = {0 :'word_freq'})
#word_freqs.iloc[50:100]
word_freqs.head(50)

,word,word_freq
0,beer,5357
1,head,4554
2,taste,3726
3,dark,3043
4,light,2678
5,good,2653
6,like,2630
7,sweet,2530
8,medium,2424
9,malt,2396


In [29]:
beer_attributes = ['dark', 'light', 'sweet', 'malt', 'carbonation', 'white','brown', 'coffee', 'caramel', 'smooth', 'bitterness', 'great', 'citrus', 'bitter', 'dry', 'vanilla', 'fruit', 'orange', 'thick', 'sweetness', 'hop', 'creamy', 'stout', 'balanced', 'rich', 'strong']

In [30]:
beer_attributes

['dark',
 'light',
 'sweet',
 'malt',
 'carbonation',
 'white',
 'brown',
 'coffee',
 'caramel',
 'smooth',
 'bitterness',
 'great',
 'citrus',
 'bitter',
 'dry',
 'vanilla',
 'fruit',
 'orange',
 'thick',
 'sweetness',
 'hop',
 'creamy',
 'stout',
 'balanced',
 'rich',
 'strong']

### Task C - Performing Similarity Analysis

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [133]:
input_attributes = input()

heavy, brown, malty


In [134]:
review_documents = beer_review_data['user_review'].tolist()
review_documents.insert(0,input_attributes)
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_reviews = tfidf_vectorizer.fit_transform(review_documents)
print(tfidf_matrix_reviews.shape)

(7379, 16481)


In [135]:
cosine_scores = cosine_similarity(tfidf_matrix_reviews[0:1], tfidf_matrix_reviews)
beer_review_data['cosine_score'] = cosine_scores[0,1:]

In [136]:
beer_review_data
cosine_scores.shape

(1, 7379)

### Task D - Performing Sentiment Analysis using Vader

In [47]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [137]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sent_analyzer = SentimentIntensityAnalyzer()

In [138]:
def cal_neg_sent_scores(review):
    return sent_analyzer.polarity_scores(review)['neg']

def cal_pos_sent_scores(review):
    return sent_analyzer.polarity_scores(review)['pos']

def cal_neu_sent_scores(review):
    return sent_analyzer.polarity_scores(review)['neu']

def cal_compound_sent_scores(review):
    return sent_analyzer.polarity_scores(review)['compound']

In [139]:
beer_review_data['sent_scores'] = beer_review_data['user_review'].apply(cal_compound_sent_scores)

In [140]:
beer_review_data[beer_review_data['sent_scores']<0]['sent_scores'].min()

-0.9743

### Task E - Providing recommendations

In [141]:
beer_review_data['final_eval_score']= (beer_review_data['cosine_score']*beer_review_data['sent_scores'])

In [142]:
beer_review_data[['prod_name','rating','final_eval_score']].groupby("prod_name").mean().sort_values(by = 'final_eval_score', ascending = False)[0:3]

,rating,final_eval_score
prod_name,,
Wee Heavy,4.160,0.078814
Indian Brown Ale,4.035,0.072076
Nillerzzzzz,4.290,0.066047


### Task F - Using Spacy to find similarity Scores

In [80]:
!pip install -U pip setuptools whe]
!pip install -U spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 2.0 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 28.0 MB/s eta 0:00:01
  Attempting uninstall: setuptools
    Found existing installation: setuptools 61.2.0
    Uninstalling setuptools-61.2.0:
      Successfully uninstalled setuptools-61.2.0
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 18.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [83]:
import spacy
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.9 MB/s eta 0:00:0000:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [86]:
nlp = spacy.load('en_core_web_lg')

In [100]:
def cal_similarity_spacy(review_string):
    doc1 = nlp(review_string)
    doc2 = nlp(input_attributes)
    similarity_score = doc1.similarity(doc2)
    return similarity_score

In [102]:
#cal_similarity_spacy(beer_review_data['user_review'][0])

beer_review_data['similarity_score_spacy']=beer_review_data['user_review'].apply(cal_similarity_spacy)

/var/folders/qy/r2hbqn1n1_5dw_z52vsv_t5h0000gn/T/ipykernel_1521/1540345233.py:4: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_score = doc1.similarity(doc2)


In [103]:
beer_review_data

,prod_name,rating,user_review,words_from_reviews,cosine_score,sent_scores,final_eval_score,similarity_score_spacy
0,Prior 8,4.50,Had this on draft 9/8/22 @ the Hopleaf.\n\nMos...,"[draft, 9, 8, 22, hopleaf, mostly, opaque, bro...",0.000000,0.7632,0.000000,0.765852
1,Prior 8,4.46,Poured from a 11.2 oz bottle into a tulip glas...,"[poured, 11, 2, oz, bottle, tulip, glass, bott...",0.000000,0.9118,0.000000,0.615833
2,Prior 8,2.45,"Look: Nice color, brown with a nice head\n\nSm...","[look, nice, color, brown, nice, head, smell, ...",0.000000,0.2748,0.000000,0.649931
3,Prior 8,4.01,Had a 750mL leftover from St Bernardus day. Be...,"[750ml, leftover, st, bernardus, day, best, ti...",0.065932,0.9882,0.065154,0.707857
4,Prior 8,4.35,Poured into a snifter a medium brown with a ta...,"[poured, snifter, medium, brown, tall, manilla...",0.089195,0.9710,0.086608,0.802203
...,...,...,...,...,...,...,...,...
7373,Northern Hemisphere Harvest Wet Hop IPA,4.30,Look is dark orange/red with a good size head ...,"[look, dark, orange, red, good, size, head, fo...",0.032090,0.8554,0.027449,0.708635
7374,Northern Hemisphere Harvest Wet Hop IPA,4.27,-optically clear golden-amber bottle pour with...,"[optically, clear, golden, amber, bottle, pour...",0.000000,0.8083,0.000000,0.694626
7375,Northern Hemisphere Harvest Wet Hop IPA,4.34,SN never disappoints. This is one of their bes...,"[sn, never, disappoints, one, best, copper, da...",0.024354,0.9725,0.023684,0.842526
7376,Northern Hemisphere Harvest Wet Hop IPA,4.77,"De Labirratorium. En copa Teku. En almuerzo, c...","[de, labirratorium, en, copa, teku, en, almuer...",0.000000,0.4588,0.000000,0.242921


### Task G - Providing recommendations using Spacy Similarity scores

In [105]:
beer_review_data['final_eval_score_using_ratings'] = beer_review_data['final_eval_score']*(beer_review_data['rating']/max(beer_review_data['rating']))

In [106]:
beer_review_data

,prod_name,rating,user_review,words_from_reviews,cosine_score,sent_scores,final_eval_score,similarity_score_spacy,final_eval_score_using_ratings
0,Prior 8,4.50,Had this on draft 9/8/22 @ the Hopleaf.\n\nMos...,"[draft, 9, 8, 22, hopleaf, mostly, opaque, bro...",0.000000,0.7632,0.000000,0.765852,0.000000
1,Prior 8,4.46,Poured from a 11.2 oz bottle into a tulip glas...,"[poured, 11, 2, oz, bottle, tulip, glass, bott...",0.000000,0.9118,0.000000,0.615833,0.000000
2,Prior 8,2.45,"Look: Nice color, brown with a nice head\n\nSm...","[look, nice, color, brown, nice, head, smell, ...",0.000000,0.2748,0.000000,0.649931,0.000000
3,Prior 8,4.01,Had a 750mL leftover from St Bernardus day. Be...,"[750ml, leftover, st, bernardus, day, best, ti...",0.065932,0.9882,0.065154,0.707857,0.052254
4,Prior 8,4.35,Poured into a snifter a medium brown with a ta...,"[poured, snifter, medium, brown, tall, manilla...",0.089195,0.9710,0.086608,0.802203,0.075349
...,...,...,...,...,...,...,...,...,...
7373,Northern Hemisphere Harvest Wet Hop IPA,4.30,Look is dark orange/red with a good size head ...,"[look, dark, orange, red, good, size, head, fo...",0.032090,0.8554,0.027449,0.708635,0.023607
7374,Northern Hemisphere Harvest Wet Hop IPA,4.27,-optically clear golden-amber bottle pour with...,"[optically, clear, golden, amber, bottle, pour...",0.000000,0.8083,0.000000,0.694626,0.000000
7375,Northern Hemisphere Harvest Wet Hop IPA,4.34,SN never disappoints. This is one of their bes...,"[sn, never, disappoints, one, best, copper, da...",0.024354,0.9725,0.023684,0.842526,0.020558
7376,Northern Hemisphere Harvest Wet Hop IPA,4.77,"De Labirratorium. En copa Teku. En almuerzo, c...","[de, labirratorium, en, copa, teku, en, almuer...",0.000000,0.4588,0.000000,0.242921,0.000000


In [109]:
beer_review_data[['prod_name','rating','final_eval_score','final_eval_score_using_ratings']].groupby("prod_name").mean().sort_values(by = 'final_eval_score_using_ratings', ascending = False)[0:3]

,rating,final_eval_score,final_eval_score_using_ratings
prod_name,,,
Sublimely Self-Righteous Black IPA,4.385625,0.057928,0.050052
Bad Boy,4.322000,0.053776,0.049263
Wookey Jack,4.135455,0.054925,0.045452
